In [1]:
! ls -alh /var/genie-in-the-box/src/conf/long-term-memory/

total 40K
drwxr-xr-x 2 1000 1000 4.0K Aug  9 19:33 .
drwxr-xr-x 4 1000 1000 4.0K Jul 19 20:04 ..
-rw-r--r-- 1 1000 1000 4.0K Jul 19 20:04 ._events.jsonl
-rw-r--r-- 1 1000 1000  16K Aug  9 19:33 events.csv
-rw-r--r-- 1 1000 1000 9.9K Aug  9 19:33 events.jsonl


In [2]:
# ! pip install python-dotenv
! pip install langchain
! pip install langchain[docarray]
! pip install tiktoken
! pip install pinecone-client
! pip install jq
! pip install chromadb
! pip install openai[embeddings]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 3.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.7/239.7 kB 38.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 13.5 MB/s eta 0:00:00
  Created wheel for hnswlib: filename=hnswlib-0.7.0-cp310-cp310-linux_x86_64.w

In [3]:
import os

In [3]:
# Import prompt and define PromptTemplate

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI( model_name="gpt-4-0613", temperature=0.0 )


In [4]:
path = "/var/genie-in-the-box/src/lib"
sys.path.append( path )
import util as du
du.add_to_path( path )
# sys.path

Path [/var/genie-in-the-box/src/lib] already in sys.path


In [5]:
# import util as du
du.print_banner( "foo?" )

------------------------------------------------------------------------------------------------------------------------
- foo?
------------------------------------------------------------------------------------------------------------------------



In [56]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory( memory_key="chat_history", return_messages=True )

In [57]:
qa = ConversationalRetrievalChain.from_llm(
    llm,
    in_memory_jsonl_index.vectorstore.as_retriever(),
    memory=memory
)

In [6]:
import pandas as pd

In [52]:
# Load CSV data From file into the Pandas DataFrame.
df = pd.read_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv" )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   start_date           200 non-null    object
 1   end_date             200 non-null    object
 2   duration             200 non-null    int64 
 3   start_time           200 non-null    object
 4   end_time             200 non-null    object
 5   event_type           200 non-null    object
 6   recurrent            200 non-null    bool  
 7   recurrence_interval  50 non-null     object
 8   description          200 non-null    object
 9   priority_level       200 non-null    object
dtypes: bool(1), int64(1), object(8)
memory usage: 14.4+ KB


### Count events by type, manually. Brute force!

In [53]:
def get_event_count( df, event ):

    print( "Counting [{}] events...".format( event ), end="\n\n" )

    count = df[ df.event_type == event ].shape[ 0 ]

    print()
    print( "Done! Total {} {} events".format( count, event ) )

get_event_count( df, "Birthday" )

Counting [Birthday] events...


Done! Total 17 Birthday events


In [58]:
from langchain.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent

llm = ChatOpenAI( model_name="gpt-4-0613", temperature=0.0 )

raw_df = pd.read_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv" )


In [65]:
template_string = """
You are an event planner with proven capacity for querying pandas dataframes.
The dataframe that I am providing you for inspection contains two important fields:
1) event_type: tells us what kind of event is taking place, such as birthday, conference, meetings, etc.
2) description: tells us who or what the event is about, as in 'my friend's birthday', the name of a conference, or what a meeting is about.
I expect you to query the dataframe provided and answer in plain English unless specifically requested to do otherwise.
NEVER display raw dataframe content, unless specifically requested to do otherwise.
All dates and temporal references like 'today', 'yesterday', 'next week' should all be calculated by using Python's datetime library.
When I ask you about events contained in this dataframe, you will always sort the events in ascending order, 1 per line, unless specifically requested to do otherwise.
When summarizing your results you will include the event type, start time, and what the event is about, using conversational English, unless specifically requested to do otherwise.
When returning a list of events do not prepend numbers to them, unless specifically requested to do otherwise.
If the question, delimited by three hash symbols ###, is vague or difficult to answer, please ask for clarification.
Given that, answer the following question: ###{question}###
"""

event_prompt_template = ChatPromptTemplate.from_template( template_string )
# print( prompt_template, end="\n\n" )

event_prompt = event_prompt_template.format_messages( question="How many records are in this dataset?" )
print( event_prompt, end="\n\n" )

# response_autoencoder = llm( prompt_autoencoder )
# print( response_autoencoder.content, end="\n\n" )

[HumanMessage(content="\nYou are an event planner with proven capacity for querying pandas dataframes.\nThe dataframe that I am providing you for inspection contains two important fields:\n1) event_type: tells us what kind of event is taking place, such as birthday, conference, meetings, etc.\n2) description: tells us who or what the event is about, as in 'my friend's birthday', the name of a conference, or what a meeting is about.\nI expect you to query the dataframe provided and answer in plain English unless specifically requested to do otherwise.\nNEVER display raw dataframe content, unless specifically requested to do otherwise.\nAll dates and temporal references like 'today', 'yesterday', 'next week' should all be calculated by using Python's datetime library.\nWhen I ask you about events contained in this dataframe, you will always sort the events in ascending order, 1 per line, unless specifically requested to do otherwise.\nWhen summarizing your results you will include the ev

In [66]:
df_agent = create_pandas_dataframe_agent( llm, raw_df, verbose=True )

In [60]:
event_prompt = event_prompt_template.format_messages( question="How many records are in this dataset?" )
df_agent.run( event_prompt )



> Entering new AgentExecutor chain...
Thought: To find out how many records are in the dataset, I can use the `shape` attribute of the dataframe. The `shape` attribute returns a tuple representing the dimensionality of the DataFrame. The first element of the tuple will give us the number of rows, which is the number of records in the dataset.
Action: python_repl_ast
Action Input: df.shape[0]
Observation: 200
Thought:I now know the final answer
Final Answer: There are 200 records in this dataset.

> Finished chain.


'There are 200 records in this dataset.'

In [68]:
# Create function to take a question and call the data frame agent.
def query_dataframe( question ):

    event_prompt = event_prompt_template.format_messages( question=question )
    response = df_agent.run( event_prompt )

    return response

response = query_dataframe( "What day is today? What day was yesterday? What day is tomorrow?" )
response

In [16]:
response = query_dataframe( "How many different types of events do I have?" )
response



> Entering new AgentExecutor chain...
Thought: To find out how many different types of events there are, I need to find the unique values in the 'event' column of the dataframe.
Action: python_repl_ast
Action Input: df['event'].nunique()
Observation: 13
Thought:I now know the final answer
Final Answer: There are 13 different types of events.

> Finished chain.


'There are 13 different types of events.'

In [17]:
response = query_dataframe( "How many TODO events do I have?" )
response



> Entering new AgentExecutor chain...
Thought: To find out how many TODO events there are, I need to filter the dataframe where the 'event' column is equal to 'TODO' and then count the number of rows.
Action: python_repl_ast
Action Input: df[df['event'] == 'TODO'].shape[0]
Observation: 21
Thought:I now know the final answer
Final Answer: You have 21 TODO events.

> Finished chain.


'You have 21 TODO events.'

In [18]:
get_event_count( df, "TODO" )

Counting [TODO] events...


Done! Total 21 TODO events


In [20]:
response = query_dataframe( "Count and display my birthday events in August" )
response



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to filter the dataframe to only include events that occur in August and have 'birthday' in their description. Then, I can count the number of these events.
Action: python_repl_ast
Action Input: df[(df['start_date'].str.contains('-08-')) & (df['description'].str.contains('birthday'))].shape[0]
Observation: AttributeError: Can only use .str accessor with string values!
Thought:The error suggests that the 'start_date' column is not of string type. I need to convert it to string before applying the str.contains() function.
Action: python_repl_ast
Action Input: df['start_date'] = df['start_date'].astype(str)
df[(df['start_date'].str.contains('-08-')) & (df['description'].str.contains('birthday'))].shape[0]
Observation: 11
Thought:I now know the final answer
Final Answer: There are 11 birthday events in August.

> Finished chain.


'There are 11 birthday events in August.'

In [69]:
response = query_dataframe( "Who has a birthday in August?" )
response



> Entering new AgentExecutor chain...
Thought: The question is asking for events of type 'birthday' that occur in August. To answer this, I need to filter the dataframe for rows where the 'event_type' is 'birthday' and the 'start_date' or 'end_date' is in August. However, I don't see any 'birthday' events in the provided dataframe head. I need to check the unique values in the 'event_type' column to confirm if there are any 'birthday' events in the dataframe.
Action: python_repl_ast
Action Input: df['event_type'].unique()
Observation: ['Conference' 'Performance' 'Concert' 'Task' 'Workout' 'TODO' 'Interview'
 'Meeting' 'Workshop' 'Anniversary' 'Birthday' 'Appointment'
 'Subscription']
Thought:The 'event_type' column does contain 'Birthday' events. Now I need to filter the dataframe for 'Birthday' events that occur in August. I will convert the 'start_date' and 'end_date' columns to datetime format and extract the month to check if it's August (8th month of the year).
Action: python_re

KeyboardInterrupt: 

In [62]:
response = query_dataframe( "What events do I have this week?" )
response



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to filter the dataframe to only include events that occur this week. I will need to use Python's datetime library to determine the dates for this week. 
Action: python_repl_ast
Action Input: import datetime
Observation: 
Thought:Now that I have imported the datetime library, I can determine the dates for this week. This week starts on the Monday of the current week and ends on the Sunday of the current week. 
Action: python_repl_ast
Action Input: today = datetime.date.today()
week_start = today - datetime.timedelta(days=today.weekday())
week_end = week_start + datetime.timedelta(days=6)
week_start, week_end
Observation: (datetime.date(2023, 8, 7), datetime.date(2023, 8, 13))
Thought:The start and end dates for this week are August 7, 2023, and August 13, 2023, respectively. Now I will filter the dataframe to only include events that start and end within this date range. 
Action: python_repl_ast
Action Inpu

"The events for this week are:\n1. Anniversary: Alice (aunt)'s anniversary\n2. Anniversary: Barbara Jane Ruiz (mother)'s anniversary\n3. Appointment: This is a appointment.\n4. Appointment: This is a appointment.\n5. Birthday: Gregorio (friend)'s birthday\n6. Birthday: Jenny (coworker)'s birthday\n7. Birthday: John (coworker)'s birthday\n8. Birthday: Pablo (friend)'s birthday\n9. Interview: This is a interview.\n10. Meeting: This is a meeting.\n11. Meeting: This is a meeting.\n12. Performance: This is a performance.\n13. TODO: This is a todo.\n14. TODO: This is a todo.\n15. TODO: This is a todo.\n16. TODO: This is a todo.\n17. TODO: This is a todo.\n18. Task: This is a task.\n19. Task: This is a task.\n20. Workout: This is a workout.\n21. Workshop: This is a workshop."

In [43]:
events = response.split( "\n\n" )[ 1: ]
for event in events: print( event )

1. Gregorio (friend)'s birthday on 2023-08-02
2. Sally (sister)'s birthday on 2023-08-02
3. John (coworker)'s birthday on 2023-08-07
4. Pablo (friend)'s birthday from 2023-08-10 to 2023-08-11
5. Jenny (coworker)'s birthday on 2023-08-12
6. Gregorio (friend)'s birthday on 2023-08-13
7. Bob (brother)'s birthday from 2023-08-14 to 2023-08-21
8. Pablo (friend)'s birthday on 2023-08-16
9. Bob (brother)'s birthday from 2023-08-22 to 2023-08-29
10. Barbara Jane Ruiz (mother)'s birthday on 2023-08-24
11. Inash (girlfriend)'s birthday from 2023-08-27 to 2023-08-30


In [44]:

response = query_dataframe( "What are your instructions?" )
response

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




> Entering new AgentExecutor chain...
Thought: The question is asking for the instructions that were given in the previous message.
Final Answer: The instructions are:
1. As an event planner, I am expected to query the provided pandas dataframe and answer in plain English unless specifically requested to do otherwise.
2. I should never display raw dataframe content, unless specifically requested to do otherwise.
3. All dates and temporal references like 'today', 'yesterday', 'next week' should all be calculated by using Python's datetime library.
4. When asked about events contained in this dataframe, I will always sort the events in ascending order, 1 per line, unless specifically requested to do otherwise.
5. If the question, delimited by three hash symbols ###, is vague or difficult to answer, I should ask for clarification.

> Finished chain.


"The instructions are:\n1. As an event planner, I am expected to query the provided pandas dataframe and answer in plain English unless specifically requested to do otherwise.\n2. I should never display raw dataframe content, unless specifically requested to do otherwise.\n3. All dates and temporal references like 'today', 'yesterday', 'next week' should all be calculated by using Python's datetime library.\n4. When asked about events contained in this dataframe, I will always sort the events in ascending order, 1 per line, unless specifically requested to do otherwise.\n5. If the question, delimited by three hash symbols ###, is vague or difficult to answer, I should ask for clarification."